## Loading dependencies

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
import keras_tuner as kt
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/MachineLearningProject")
import sys
sys.path.append("/content/drive/MyDrive/MachineLearningProject")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating the models

In [ ]:
%run sequential.py

## Loading the data

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/train",
                                                       label_mode="binary",
                                                       color_mode="rgb",
                                                       batch_size=32,
                                                       image_size=(224, 224),
                                                       shuffle=True,
                                                       seed=42)

val_ds = tf.keras.utils.image_dataset_from_directory("/content/drive/MyDrive/MachineLearningProject/data/test",
                                                     label_mode="binary",
                                                     color_mode="rgb",
                                                     batch_size=32,
                                                     image_size=(224, 224),
                                                     shuffle=True,
                                                     seed=42)

Found 4743 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.


## Sequential 1

### Loading the model

In [ ]:
seq_1 = tf.keras.models.load_model(
    "/content/drive/MyDrive/MachineLearningProject/models/sequential/seq_1.keras",
    safe_mode=False)

seq_1.summary()

Model: "Sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Random_horizontal_flip (Ra  (None, 224, 224, 3)       0         
 ndomFlip)                                                       
                                                                 
 Random_contrast (RandomCon  (None, 224, 224, 3)       0         
 trast)                                                          
                                                                 
 Per_image_standardisation   (None, 224, 224, 3)       0         
 (Lambda)                                                        
                                                                 
 Conv_1 (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 MaxPool_1 (MaxPooling2D)    (None, 112, 112, 32)      0         
                                                      

### Compiling the model

In [ ]:
seq_1.compile(loss="binary_crossentropy",
              optimizer=Adam(),
              metrics=["accuracy"])

### Creating callbacks

In [ ]:
seq_1_checkpoint = ModelCheckpoint(
    filepath="/content/drive/MyDrive/MachineLearningProject/model_weights/sequential/seq_1.keras",
    save_weights_only=True,
    save_best_only=False)

seq_1_csv_logger = CSVLogger('/content/drive/MyDrive/MachineLearningProject/histories/sequential/seq_1.log')

### Fitting the model

In [ ]:
seq_1_history = seq_1.fit(train_ds,
                          epochs=50,
                          steps_per_epoch=len(train_ds),
                          validation_data=val_ds,
                          validation_steps=len(val_ds),
                          callbacks=[seq_1_checkpoint, seq_1_csv_logger])

Epoch 1/50
149/149 [==============================] - 381s 2s/step - loss: 0.4915 - accuracy: 0.7592 - val_loss: 0.3643 - val_accuracy: 0.8505
Epoch 2/50
149/149 [==============================] - 13s 81ms/step - loss: 0.3552 - accuracy: 0.8503 - val_loss: 0.4209 - val_accuracy: 0.8319
Epoch 3/50
149/149 [==============================] - 12s 80ms/step - loss: 0.2995 - accuracy: 0.8741 - val_loss: 0.3079 - val_accuracy: 0.8691
Epoch 4/50
149/149 [==============================] - 12s 79ms/step - loss: 0.2438 - accuracy: 0.9005 - val_loss: 0.2731 - val_accuracy: 0.8834
Epoch 5/50
149/149 [==============================] - 12s 80ms/step - loss: 0.1953 - accuracy: 0.9201 - val_loss: 0.2813 - val_accuracy: 0.8902
Epoch 6/50
149/149 [==============================] - 12s 80ms/step - loss: 0.1756 - accuracy: 0.9283 - val_loss: 0.2680 - val_accuracy: 0.8953
Epoch 7/50
149/149 [==============================] - 12s 80ms/step - loss: 0.1467 - accuracy: 0.9467 - val_loss: 0.3199 - val_accuracy: 

## Sequential 2

### Loading the model

In [ ]:
seq_2 = tf.keras.models.load_model(
    "/content/drive/MyDrive/MachineLearningProject/models/sequential/seq_2.keras",
    safe_mode=False)

seq_2.summary()

Model: "Sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Random_horizontal_flip (Ra  (None, 224, 224, 3)       0         
 ndomFlip)                                                       
                                                                 
 Random_contrast (RandomCon  (None, 224, 224, 3)       0         
 trast)                                                          
                                                                 
 Per_image_standardisation   (None, 224, 224, 3)       0         
 (Lambda)                                                        
                                                                 
 Conv_1 (Conv2D)             (None, 224, 224, 128)     3584      
                                                                 
 ReLU_1 (ReLU)               (None, 224, 224, 128)     0         
                                                      

### Hyperparameter tuning

In [ ]:
hp_model = tf.keras.models.load_model(
        "/content/drive/MyDrive/MachineLearningProject/models/sequential/seq_2.keras",
        safe_mode=False)

def model_builder(hp, model=hp_model):

  # Learning Rate
  hp_learning_rate = hp.Choice(name="learning_rate",
                               values = [0.01, 0.0075, 0.005, 0.0025, 0.001, 0.0005])

  # Compiling the model
  model.compile(loss="binary_crossentropy",
                optimizer=Adam(learning_rate=hp_learning_rate),
                metrics=["accuracy"])

  return model

In [ ]:
tuner = kt.BayesianOptimization(model_builder,
                                objective="val_accuracy",
                                max_trials=10,
                                directory="content/drive/MyDrive/MachineLearningProject/tuner",
                                project_name="lr_tuner")

tuner.search_space_summary(train_ds)

Search space summary
Default search space size: 1
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0075, 0.005, 0.0025, 0.001, 0.0005], 'ordered': True}


In [ ]:
tuner.search(train_ds,
             epochs=10,
             validation_data=val_ds)

best_hyperparameters = tuner.get_best_hyperparameters()[0]
best_hyperparameters.values

Trial 10 Complete [00h 02m 07s]
val_accuracy: 0.9476351141929626

Best val_accuracy So Far: 0.9476351141929626
Total elapsed time: 00h 21m 17s


{'learning_rate': 0.001}

In [ ]:
best_lr = best_hyperparameters.values["learning_rate"]

### Compiling the model

In [ ]:
seq_2.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=best_lr),
              metrics=["accuracy"])

### Creating callbacks

In [ ]:
seq_2_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/MachineLearningProject/model_weights/sequential/seq_2.keras",
    save_weights_only=True,
    save_best_only=False)

seq_2_csv_logger = CSVLogger('/content/drive/MyDrive/MachineLearningProject/histories/sequential/seq_2.log')

### Fitting the model

In [ ]:
seq_2_history = seq_2.fit(train_ds,
                          epochs=50,
                          steps_per_epoch=len(train_ds),
                          validation_data=val_ds,
                          validation_steps=len(val_ds),
                          callbacks=[seq_2_checkpoint, seq_2_csv_logger])

Epoch 1/50
149/149 [==============================] - 17s 80ms/step - loss: 0.4303 - accuracy: 0.8100 - val_loss: 0.4492 - val_accuracy: 0.8100
Epoch 2/50
149/149 [==============================] - 13s 83ms/step - loss: 0.3307 - accuracy: 0.8670 - val_loss: 0.3813 - val_accuracy: 0.8674
Epoch 3/50
149/149 [==============================] - 13s 82ms/step - loss: 0.2835 - accuracy: 0.8836 - val_loss: 0.3019 - val_accuracy: 0.8860
Epoch 4/50
149/149 [==============================] - 13s 82ms/step - loss: 0.2446 - accuracy: 0.8975 - val_loss: 0.2673 - val_accuracy: 0.8834
Epoch 5/50
149/149 [==============================] - 13s 81ms/step - loss: 0.2149 - accuracy: 0.9117 - val_loss: 0.2496 - val_accuracy: 0.9054
Epoch 6/50
149/149 [==============================] - 13s 82ms/step - loss: 0.1980 - accuracy: 0.9165 - val_loss: 0.2614 - val_accuracy: 0.9046
Epoch 7/50
149/149 [==============================] - 13s 81ms/step - loss: 0.1828 - accuracy: 0.9260 - val_loss: 0.2589 - val_accuracy: